In [1]:
import requests
import zipfile
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import json

In [ ]:
import pandas as pd

# Define file paths for the datasets
datasets = {
    "ER_H2O_IWRMD_FI": "C:/Users/rohit/OneDrive/Documents/DAEN690/New Datasets/ER_H2O_IWRMD_FI.xlsx",
    "GF_XPD_GBPC": "C:/Users/rohit/OneDrive/Documents/DAEN690/New Datasets/GF_XPD_GBPC.xlsx",
    "IU_COR_BRIB": "C:/Users/rohit/OneDrive/Documents/DAEN690/New Datasets/IU_COR_BRIB.xlsx",
    "VC_IHR_PSRC": "C:/Users/rohit/OneDrive/Documents/DAEN690/New Datasets/VC_IHR_PSRC.xlsx",
    "VC_PRS_UNSNT": "C:/Users/rohit/OneDrive/Documents/DAEN690/New Datasets/VC_PRS_UNSNT.xlsx",
    "VC_SNS_WALN_DRK": "C:/Users/rohit/OneDrive/Documents/DAEN690/New Datasets/VC_SNS_WALN_DRK.xlsx",
    #"GF_FRN_FDI": "C:/Users/rohit/OneDrive/Documents/DAEN690/New Datasets/GF_FRN_FDI.xlsx"
}

# Read datasets into DataFrames
dataframes = {key: pd.read_excel(path) for key, path in datasets.items()}

# Get available years dynamically from each dataset
available_years = {key: [col for col in df.columns if col.isdigit()] for key, df in dataframes.items()}

# Ensure required columns exist before selecting
water_columns = [col for col in ["GeoAreaName", "SeriesCode", "Indicator"] + available_years["ER_H2O_IWRMD_FI"] if col in dataframes["ER_H2O_IWRMD_FI"].columns]
df_water_mgmt = dataframes["ER_H2O_IWRMD_FI"][water_columns] if water_columns else pd.DataFrame()

# Standardize Indicator naming convention across datasets
if not df_water_mgmt.empty:
    df_water_mgmt.loc[:, "Indicator"] = "6.5.1"
    df_water_mgmt.loc[:, "Indicator_Type"] = "Water Management"
    df_water_mgmt.loc[:, "Units"] = "PERCENT"  # Assign default unit
    df_water_mgmt.fillna(0, inplace=True)  # Replace null values with 0

df_unsentenced = dataframes["VC_PRS_UNSNT"][[col for col in ["GeoAreaName", "SeriesCode", "Indicator"] + available_years["VC_PRS_UNSNT"] if col in dataframes["VC_PRS_UNSNT"].columns]]
df_unsentenced["Indicator_Type"] = "Unsentenced Detainees"
df_unsentenced["Units"] = "PERCENT"  # Assign default unit

df_homicide = dataframes["VC_IHR_PSRC"][[col for col in ["GeoAreaName", "SeriesCode", "Indicator", "Units"] + available_years["VC_IHR_PSRC"] if col in dataframes["VC_IHR_PSRC"].columns]]
df_homicide["Indicator_Type"] = "Homicide"

# Keep homicide values in original PER_100000_POP and retain the Units column
def restructure_data(df):
    """Transforms the dataset to have years as columns and adds an Indicator column."""
    if df.empty:
        return pd.DataFrame()
    
    id_vars = ["GeoAreaName", "SeriesCode", "Indicator", "Indicator_Type", "Units"]
    df_long = df.melt(id_vars=id_vars, var_name="Year", value_name="Value")
    df_long["Value"] = df_long["Value"].fillna(0)  # Replace empty values with 0
    return df_long

# Transform individual datasets
df_water_mgmt_long = restructure_data(df_water_mgmt)
df_unsentenced_long = restructure_data(df_unsentenced)
df_homicide_long = restructure_data(df_homicide)

# Combine all indicators
df_combined = pd.concat([df_water_mgmt_long, df_unsentenced_long, df_homicide_long], ignore_index=True)

# Pivot the data to make years as columns
index_cols = ["GeoAreaName", "Indicator", "Indicator_Type", "SeriesCode", "Units"]
df_pivot = df_combined.pivot_table(index=index_cols, columns="Year", values="Value", aggfunc="sum").reset_index()

# Remove column name from pivot table
df_pivot.columns.name = None

# Rename GeoAreaName to Country
df_pivot.rename(columns={"GeoAreaName": "Country"}, inplace=True)

# Add SDG Goal column
df_pivot["SDG_Goal"] = "16 & 6 (Peace, Justice, Water Management)"

df_pivot = df_pivot.fillna(0)

# Save the result
df_pivot.to_csv("output.csv", index=False)




C:\Users\rohit\AppData\Local\Temp\ipykernel_39664\3915360713.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_water_mgmt.loc[:, "Indicator_Type"] = "Water Management"
C:\Users\rohit\AppData\Local\Temp\ipykernel_39664\3915360713.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_water_mgmt.loc[:, "Units"] = "PERCENT"  # Assign default unit
C:\Users\rohit\AppData\Local\Temp\ipykernel_39664\3915360713.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame


In [ ]:
import pandas as pd

# Load the output.csv file to understand its structure
output_csv_path = "C:/Users/rohit/Downloads/output.csv"
output_df = pd.read_csv(output_csv_path)

# Load the Excel files
excel_files = {
    "IU_COR_BRIB": "C:/Users/rohit/OneDrive/Documents/DAEN690/Newer Datasets/IU_COR_BRIB (1).xlsx",
    "GF_XPD_GBPC": "C:/Users/rohit/OneDrive/Documents/DAEN690/Newer Datasets/GF_XPD_GBPC (1).xlsx",
    "EN_WWT_WWDS": "C:/Users/rohit/OneDrive/Documents/DAEN690/Newer Datasets/EN_WWT_WWDS.xlsx",
    "EN_H2O_OPAMBQ": "C:/Users/rohit/OneDrive/Documents/DAEN690/Newer Datasets/EN_H2O_OPAMBQ.xlsx",
    "DC_TOF_WASHL": "C:/Users/rohit/OneDrive/Documents/DAEN690/Newer Datasets/DC_TOF_WASHL.xlsx"
}

# Read each sheet in the Excel files
excel_data = {name: pd.ExcelFile(path) for name, path in excel_files.items()}

# Define column mappings
column_mappings = {
    "GeoAreaName": "Country",
    "SeriesDescription": "Indicator_Type",
}

# Standardize each dataset
standardized_dfs = []
for name, excel_file in excel_data.items():     
    sheet_name = excel_file.sheet_names[0]  # Assume first sheet is relevant
    df = excel_file.parse(sheet_name)  # Read first sheet
    df = df.rename(columns=column_mappings)
    
    # Select relevant columns
    relevant_columns = ["Country", "Indicator", "SeriesCode", "Indicator_Type", "Units"] + \
                       [str(year) for year in range(2000, 2024)]
    
    # Ensure missing columns are added and in correct order
    for col in relevant_columns:
        if col not in df.columns:
            df[col] = None

    df = df[relevant_columns]  # Reorder columns
    standardized_dfs.append(df)

# Concatenate all datasets into a single DataFrame
merged_df = pd.concat([output_df] + standardized_dfs, ignore_index=True)

# Fill missing values with 0
merged_df = merged_df.fillna(0) 

# Remove 'SDG_Goal' column if it exists
if 'SDG_Goal' in merged_df.columns:
    merged_df = merged_df.drop(columns=['SDG_Goal'])

# Sort the dataset by Country
sorted_final_df = merged_df.sort_values(by=["Country"]).reset_index(drop=True)

# Save the sorted dataset
sorted_output_path = "C:/Users/rohit/Downloads/final_integrated_output.csv"
sorted_final_df.to_csv(sorted_output_path, index=False)

print(f"Sorted and integrated dataset saved to: {sorted_output_path}")


Sorted and integrated dataset saved to: C:/Users/rohit/Downloads/final_integrated_output.csv


C:\Users\rohit\AppData\Local\Temp\ipykernel_31552\316943551.py:45: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_df = pd.concat([output_df] + standardized_dfs, ignore_index=True)


: 

In [ ]:
import pandas as pd

# Step 1: Load the original SDG 6 and 16 dataset (already in wide format)
sorted_df = pd.read_csv("C:/Users/rohit/OneDrive/Documents/DAEN690/Dataset_Sorted_by_Indicator.csv")

# Step 2: Load SDG 17 dataset (raw format)
sdg17_df = pd.read_excel("C:/Users/rohit/OneDrive/Documents/DAEN690/Newer Datasets/GF_FRN_FDI.xlsx")

# Step 3: Melt SDG 17 into long format
sdg17_long = sdg17_df.melt(
    id_vars=["Goal", "Target", "Indicator", "SeriesCode", "SeriesDescription", 
             "GeoAreaCode", "GeoAreaName", "Reporting Type", "Units"],
    var_name="TimePeriod",
    value_name="Value"
)

# Step 4: Pivot SDG 17 back into wide format with year columns
sdg17_pivot = sdg17_long.pivot_table(
    index=["GeoAreaName", "Indicator", "SeriesCode", "Units"],
    columns="TimePeriod",
    values="Value"
).reset_index()

# Step 5: Rename and format to match Dataset_Sorted_by_Indicator.csv
sdg17_pivot = sdg17_pivot.rename(columns={"GeoAreaName": "Country"})
sdg17_pivot["Indicator_Type"] = "Quantitative"  # or any default as needed

# Step 6: Ensure column consistency
expected_columns = ['Country', 'Indicator', 'Indicator_Type', 'SeriesCode', 'Units'] + \
                   [str(year) for year in range(2000, 2024)]

# Add missing columns (years) with NA
for col in expected_columns:
    if col not in sdg17_pivot.columns:
        sdg17_pivot[col] = pd.NA

# Reorder columns
sdg17_pivot = sdg17_pivot[expected_columns]

# Step 7: Concatenate SDG 17 to the original dataset
combined_dataset = pd.concat([sorted_df, sdg17_pivot], ignore_index=True)

# Step 8: Final cleanup — reset index and sort by Indicator and Country
final_dataset = combined_dataset.reset_index(drop=True)
final_dataset_sorted = final_dataset.sort_values(by=["Indicator", "Country"]).reset_index(drop=True)

# Output: final_dataset_sorted is your complete clean dataset
print(final_dataset_sorted.head(10))

# Save the sorted dataset
sorted_output_path = "C:/Users/rohit/OneDrive/Documents/DAEN690/final_dataset_sdg61617" \
".csv"
sorted_final_df.to_csv(sorted_output_path, index=False)


               Country Indicator Indicator_Type   SeriesCode           Units  \
0          Afghanistan    16.1.1       Homicide  VC_IHR_PSRC  PER_100000_POP   
1              Albania    16.1.1       Homicide  VC_IHR_PSRC  PER_100000_POP   
2              Algeria    16.1.1       Homicide  VC_IHR_PSRC  PER_100000_POP   
3       American Samoa    16.1.1       Homicide  VC_IHR_PSRC  PER_100000_POP   
4              Andorra    16.1.1       Homicide  VC_IHR_PSRC  PER_100000_POP   
5               Angola    16.1.1       Homicide  VC_IHR_PSRC  PER_100000_POP   
6             Anguilla    16.1.1       Homicide  VC_IHR_PSRC  PER_100000_POP   
7  Antigua and Barbuda    16.1.1       Homicide  VC_IHR_PSRC  PER_100000_POP   
8            Argentina    16.1.1       Homicide  VC_IHR_PSRC  PER_100000_POP   
9              Armenia    16.1.1       Homicide  VC_IHR_PSRC  PER_100000_POP   

    2000   2001   2002   2003   2004  ...   2014   2015   2016   2017   2018  \
0   0.00   0.00   0.00   0.00   0.00  .